In [3]:
import torch

In [5]:
model = torch.load('../models/best.pth')

In [7]:
model.keys()

dict_keys(['date', 'last_epoch', 'model', 'criterion', 'postprocessor', 'ema', 'scaler', 'optimizer', 'lr_warmup_scheduler', 'model_info', 'model_meta', 'model_files'])

In [12]:
model['model_meta']

{'classes': [{'title': 'horse',
   'description': '',
   'shape': 'rectangle',
   'color': '#FE9A03',
   'geometry_config': {},
   'id': 99044,
   'hotkey': ''},
  {'title': 'horse head',
   'description': '',
   'shape': 'rectangle',
   'color': '#AEFF01',
   'geometry_config': {},
   'id': 99045,
   'hotkey': ''},
  {'title': 'number plate',
   'description': '',
   'shape': 'rectangle',
   'color': '#6200FF',
   'geometry_config': {},
   'id': 99046,
   'hotkey': ''},
  {'title': 'rider',
   'description': '',
   'shape': 'rectangle',
   'color': '#02C2FF',
   'geometry_config': {},
   'id': 99047,
   'hotkey': ''},
  {'title': 'yellow stick',
   'description': '',
   'shape': 'rectangle',
   'color': '#F6F016',
   'geometry_config': {},
   'id': 99048,
   'hotkey': ''},
  {'title': 'white stick',
   'description': '',
   'shape': 'rectangle',
   'color': '#FF03D6',
   'geometry_config': {},
   'id': 99049,
   'hotkey': ''}],
 'tags': [{'name': 'Completed',
   'value_type': 'none',


In [14]:
model['model'].keys()

odict_keys(['backbone.stem.stem1.conv.weight', 'backbone.stem.stem1.bn.weight', 'backbone.stem.stem1.bn.bias', 'backbone.stem.stem1.bn.running_mean', 'backbone.stem.stem1.bn.running_var', 'backbone.stem.stem1.bn.num_batches_tracked', 'backbone.stem.stem2a.conv.weight', 'backbone.stem.stem2a.bn.weight', 'backbone.stem.stem2a.bn.bias', 'backbone.stem.stem2a.bn.running_mean', 'backbone.stem.stem2a.bn.running_var', 'backbone.stem.stem2a.bn.num_batches_tracked', 'backbone.stem.stem2b.conv.weight', 'backbone.stem.stem2b.bn.weight', 'backbone.stem.stem2b.bn.bias', 'backbone.stem.stem2b.bn.running_mean', 'backbone.stem.stem2b.bn.running_var', 'backbone.stem.stem2b.bn.num_batches_tracked', 'backbone.stem.stem3.conv.weight', 'backbone.stem.stem3.bn.weight', 'backbone.stem.stem3.bn.bias', 'backbone.stem.stem3.bn.running_mean', 'backbone.stem.stem3.bn.running_var', 'backbone.stem.stem3.bn.num_batches_tracked', 'backbone.stem.stem4.conv.weight', 'backbone.stem.stem4.bn.weight', 'backbone.stem.stem4

In [19]:
import sys
sys.path.append('../')
sys.path.append('../deim')
from deim.supervisely_integration.export import export_onnx, export_tensorrt

In [2]:
import onnxruntime as ort
import numpy as np
from PIL import Image

sess = ort.InferenceSession("../models/best_new_output.onnx", providers=["CPUExecutionProvider"])

img = np.random.rand(1,3,640,640).astype(np.float32)
outputs = sess.run(None, {"images": img})
print([o.shape for o in outputs])
print([o.dtype for o in outputs])


[(1, 300), (1, 300, 4), (1, 300)]
[dtype('int64'), dtype('float32'), dtype('float32')]


In [2]:
# check_pytorch_vs_onnx.py
import torch, onnxruntime as ort, numpy as np
from PIL import Image
import torchvision.transforms as T
import cv2
import sys
sys.path.insert(0, "..")  # поправь, если нужно, чтобы импортировать deim и cfg

# --- ЗАГРУЗКА PyTorch модели (тот же wrapper, который ты экспортировал) ---
from convert_new_output import SingleInputDFINE, load_deim_model  # путь под твой проект
cfg = load_deim_model("../models/best.pth", "../models/model_config.yml")
model_pt = SingleInputDFINE(cfg)
model_pt.eval()
device = torch.device("cpu")  # или "cuda" если хочешь
model_pt.to(device)

# --- ЗАГРУЗКА ONNX ---
onnx_path = "../models/fix_rgb.onnx"
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

# --- ЧИТАЕМ КАДР (тот же файл, что в DeepStream) ---
img_path = "../data/test_video_640.avi"  # если видео, извлечём центральный кадр
cap = cv2.VideoCapture(img_path)
ret, frame = cap.read()
if not ret:
    raise SystemExit("Не удалось прочитать видео/кадр")
frame = cv2.resize(frame, (640,640))

# two preprocessing variants:
# A) PyTorch-style (обычно): RGB, 0..1, normalized mean/std (ImageNet)
mean = [0.485,0.456,0.406]; std=[0.229,0.224,0.225]
transform_pt = T.Compose([
    T.ToTensor(),  # HWC BGR->RGB + /255
    T.Normalize(mean, std),
])
img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
inp_pt = transform_pt(Image.fromarray(img_rgb)).unsqueeze(0).to(device)

# B) DeepStream-style raw: NCHW float32, BGR, values 0..255 (без норм)
inp_ds = frame.transpose(2,0,1).astype(np.float32)[None, :, :, :]  # NCHW, BGR, 0..255

# --- Forward PyTorch (вариант PyTorch-preprocessed) ---
with torch.no_grad():
    out_pt = model_pt(inp_pt)   # ожидается dict {"labels","boxes","scores"} или tuple
# normalize output printing
def print_out(which, labels, boxes, scores):
    print(f"--- {which} ---")
    print("labels:", labels.shape, type(labels), labels.dtype if hasattr(labels,'dtype') else None)
    print("scores min/max:", float(scores.min()), float(scores.max()))
    print("boxes sample:", boxes[0,:5,:] if boxes.shape[1] >=5 else boxes)

# PyTorch outputs might be dict or tuple
if isinstance(out_pt, dict):
    labels_pt = out_pt["labels"].cpu().numpy()
    boxes_pt  = out_pt["boxes"].cpu().numpy()
    scores_pt = out_pt["scores"].cpu().numpy()
else:
    labels_pt, boxes_pt, scores_pt = out_pt
    labels_pt = labels_pt.cpu().numpy(); boxes_pt = boxes_pt.cpu().numpy(); scores_pt = scores_pt.cpu().numpy()

print_out("PyTorch (train preprocessing)", labels_pt, boxes_pt, scores_pt)

# --- ONNXRuntime: test both types of inputs ---
# 1) feed ONNX the same PyTorch-preprocessed tensor (converted to numpy float32)
onnx_in1 = inp_pt.cpu().numpy().astype(np.float32)
outs1 = sess.run(None, {"images": onnx_in1})
print_out("ONNX (fed PyTorch-normalized input)", outs1[0], outs1[1], outs1[2])

# 2) feed ONNX the raw DeepStream-like input (BGR, 0..255)
outs2 = sess.run(None, {"images": inp_ds.astype(np.float32)})
print_out("ONNX (fed raw BGR 0..255 input)", outs2[0], outs2[1], outs2[2])

# --- Quick numerical diffs between PyTorch and ONNX when fed same normalized input ---
def compare(a,b,name):
    a = np.asarray(a); b = np.asarray(b)
    print(f"{name}: shapes {a.shape} vs {b.shape}, max_abs_diff={np.max(np.abs(a - b))}, mean_abs_diff={np.mean(np.abs(a-b))}")
compare(labels_pt, outs1[0], "labels")
compare(boxes_pt, outs1[1], "boxes")
compare(scores_pt, outs1[2], "scores")


Loaded stage1 B4 HGNetV2 from local file.
Model weights loaded successfully
--- PyTorch (train preprocessing) ---
labels: (1, 300) <class 'numpy.ndarray'> int32
scores min/max: 0.008348156698048115 0.9696156978607178
boxes sample: [[445.72693 225.77625 486.1518  310.33176]
 [471.21823 288.22754 506.16565 360.8342 ]
 [310.03482 255.89984 338.69263 345.26917]
 [445.7116  225.97214 461.43457 259.25323]
 [142.62186 213.95584 175.75197 296.5071 ]]
--- ONNX (fed PyTorch-normalized input) ---
labels: (1, 300) <class 'numpy.ndarray'> float32
scores min/max: 0.002884089946746826 0.007871955633163452
boxes sample: [[ 2.0198050e+01  2.7025223e-01  3.1052227e+01  2.8351231e+01]
 [ 6.2134650e+02  4.6744752e+00  6.3274432e+02  3.2005676e+01]
 [ 6.1201001e+02 -3.6549568e+00  6.1986163e+02  3.5530586e+01]
 [ 6.2295416e+02  1.8584251e-01  6.3579016e+02  2.6900702e+01]
 [ 6.1940942e+02  2.6873493e+00  6.3276849e+02  3.2384315e+01]]
--- ONNX (fed raw BGR 0..255 input) ---
labels: (1, 300) <class 'numpy.n

In [3]:
# быстро узнать имена output'ов ONNX
import onnx
m = onnx.load("../models/fix_rgb.onnx")
for o in m.graph.output:
    print(o.name)


labels
boxes
scores


In [1]:
# run_trt_engine.py
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import cv2

TRT_LOGGER = trt.Logger(trt.Logger.INFO)
def load_engine(engine_path):
    with open(engine_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

engine = load_engine("../models/fix_rgb.engine")
context = engine.create_execution_context()

# prepare input (read same frame)
frame = cv2.imread("../data/test_video_640_frame10.jpg")  # подготовь один png 640x640 (тот же кадр)
frame = cv2.resize(frame, (640,640))
# DeepStream дает BGR 0..255 NCHW, model expects that (we embedded preprocessing)
inp = frame.transpose(2,0,1).astype(np.float32)[None, ...]

# allocate buffers
inputs, outputs, bindings, stream = [], [], [], cuda.Stream()
for i in range(engine.num_bindings):
    name = engine.get_binding_name(i)
    shape = context.get_binding_shape(i)
    dtype = trt.nptype(engine.get_binding_dtype(i))
    size = int(np.prod(shape))
    host_mem = cuda.pagelocked_empty(size, dtype)
    dev_mem = cuda.mem_alloc(host_mem.nbytes)
    bindings.append(int(dev_mem))
    if engine.binding_is_input(i):
        inputs.append((name, host_mem, dev_mem, shape, dtype))
    else:
        outputs.append((name, host_mem, dev_mem, shape, dtype))

# copy input
np.copyto(inputs[0][1], inp.ravel())
cuda.memcpy_htod_async(inputs[0][2], inputs[0][1], stream)

# execute
context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
# copy outputs
for name, host_mem, dev_mem, shape, dtype in outputs:
    cuda.memcpy_dtoh_async(host_mem, dev_mem, stream)

stream.synchronize()

# reshape and print
for name, host_mem, dev_mem, shape, dtype in outputs:
    arr = np.array(host_mem).reshape(shape)
    print("OUTPUT", name, arr.dtype, arr.shape)
    print("first 20:", arr.ravel()[:20])


: 